In [ ]:
import os, sys

# if os.path.exists('analysis'):
#     os.system('rm -rf analysis')

# !git clone https://github.com/element-fi/analysis.git

parent_dir = os.path.join(os.getcwd(), os.pardir)
sys.path.insert(1, os.path.join(parent_dir, "src"))

import json, numbers, math, time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from elfpy.simulators import YieldSimulator
from elfpy.utils.data import format_trades

pd.set_option("float_format",'{:,.4f}'.format)

In [ ]:
from elfpy.pricing_models import YieldSpacev2PricingModel

In [ ]:
target_liquidity_usd = 10*1000 # $10k
market_price = 1
apy = 0.05 # in percent
days_remaining = 365/2
time_stretch = 22.58
mdl = YieldSpacev2PricingModel(verbose=True)
(base_asset_reserves, token_asset_reserves, total_liquidity) = mdl.calc_liquidity(target_liquidity_usd=target_liquidity_usd, \
    market_price=market_price, apy=apy, days_remaining=days_remaining, time_stretch=time_stretch)
[base_asset_reserves_usd, token_asset_reserves_usd, total_liquidity_usd] = [x * market_price for x in [base_asset_reserves, token_asset_reserves, total_liquidity]]
total_supply = base_asset_reserves + token_asset_reserves
time_remaining = mdl.days_to_time_remaining(days_remaining, time_stretch)
spot_price = mdl.calc_spot_price_from_reserves(base_asset_reserves, token_asset_reserves, total_supply, time_remaining)
calc_apy = mdl.calc_apy_from_reserves(base_asset_reserves, token_asset_reserves, total_supply, time_remaining, time_stretch)
calc_hpr = calc_apy * days_remaining / 365
print(f"input: target_liquidity_usd={target_liquidity_usd}, market_price={market_price}, apy={apy}, days_remaining={days_remaining}, time_stretch={time_stretch}")
# print(f"spot price = {spot_price}")
# print('output in ETH terms:')
# print(f"base_asset_reserves: {base_asset_reserves}\ntoken_asset_reserves: {token_asset_reserves}\ntotal_liquidity: {total_liquidity}")
# print('output in USD terms:')
print(f"\
base_asset_reserves:  ${base_asset_reserves_usd:6,.0f}\n\
token_asset_reserves: ${token_asset_reserves_usd*spot_price:6,.0f} ({token_asset_reserves_usd:,.0f} PTs at {spot_price:.4f})\n\
total_liquidity:      ${total_liquidity_usd:6,.0f} ({base_asset_reserves_usd+token_asset_reserves_usd:,.2f} tokens, base + PT)\n\
APR:                   {calc_apy:6,.4%}"
)
initial_price = spot_price
initial_days_remaining = days_remaining
initial_apy = calc_apy
initial_hpr = calc_hpr
initial_base_asset_reserves = base_asset_reserves
initial_token_asset_reserves = token_asset_reserves

In [ ]:
trade_in_usd = 1000
token_out = 'fyt'
token_in = 'fyt' if token_out == 'base' else 'base'
in_ = trade_in_usd / market_price
total_supply = base_asset_reserves + token_asset_reserves
in_reserves = token_asset_reserves + total_supply if token_out == 'base' else base_asset_reserves
out_reserves = base_asset_reserves if token_out == 'base' else token_asset_reserves + total_supply
fee_percent = 0.1
time_remaining = mdl.days_to_time_remaining(days_remaining, time_stretch)
(without_fee_or_slippage, with_fee, without_fee, fee) = mdl.calc_out_given_in(in_=in_,in_reserves=in_reserves,out_reserves=out_reserves,token_out=token_out,fee_percent=fee_percent,time_remaining=time_remaining,share_price=1,init_share_price=1)
[without_fee_or_slippage_usd, with_fee_usd, without_fee_usd, fee_usd] = [x * market_price for x in [without_fee_or_slippage, with_fee, without_fee, fee]]
new_base_asset_reserves = base_asset_reserves - with_fee if token_out == 'base' else base_asset_reserves + in_
new_token_asset_reserves = token_asset_reserves + in_ if token_out == 'base' else token_asset_reserves - with_fee
new_total_supply = new_base_asset_reserves + new_token_asset_reserves
new_spot_price = mdl.calc_spot_price_from_reserves(new_base_asset_reserves, new_token_asset_reserves, new_total_supply, time_remaining)
new_total_liquidity = new_base_asset_reserves + new_token_asset_reserves * new_spot_price
[new_base_asset_reserves_usd, new_token_asset_reserves_usd, new_total_liquidity_usd] = [x * market_price for x in [new_base_asset_reserves, new_token_asset_reserves, new_total_liquidity]]
new_apy = mdl.calc_apy_from_reserves(new_base_asset_reserves, new_token_asset_reserves, new_total_supply, time_remaining, time_stretch)
print(f"trade:\n\
  in:                       {in_*market_price:8,.2f} {token_in}\n\
  without_fee_or_slippage:  {without_fee_or_slippage_usd:8,.2f} {token_out}\n\
  without_fee:              {without_fee_usd:8,.2f} {token_out}\n\
  slippage:                 {without_fee_or_slippage_usd-without_fee_usd:8,.2f} {token_out}\n\
  fee:                      {fee_usd:8,.2f} {token_out} (fee applied to {fee_usd/fee_percent:,.2f})\n\
  with_fee:                 {with_fee_usd:8,.2f} {token_out}")
print(f"new reserves:\n\
  base_asset_reserves:  ${new_base_asset_reserves_usd:6,.0f}\n\
  token_asset_reserves: ${new_token_asset_reserves_usd*new_spot_price:6,.0f} ({new_token_asset_reserves_usd:,.0f} PTs at {new_spot_price:.4f})\n\
  total_liquidity:      ${new_total_liquidity_usd:6,.0f} ({new_base_asset_reserves_usd+new_token_asset_reserves_usd:,.2f} tokens, base + PT)\n\
  APR:                   {new_apy:6,.4%}")


In [ ]:
###############################
############ SETUP ############
spot_price = initial_price
result = []
day_interval = 1/24/2
###############################
for trade_in_usd in np.concatenate([np.arange(50,1050,50),np.arange(1100,5000,100)]):
# for trade_in_usd in np.arange(1000,1001,1):
    day_bump = -day_interval
    new_base_asset_reserves = initial_base_asset_reserves + trade_in_usd / market_price
    # token_asset_reserves = token_asset_reserves - with_fee # using out amount from yieldspace trade above
    new_token_asset_reserves = initial_token_asset_reserves - trade_in_usd / market_price / spot_price # same result using no slippage or fee
    total_supply = new_base_asset_reserves + new_token_asset_reserves
    starting_time_remaining = mdl.days_to_time_remaining(initial_days_remaining, time_stretch)
    starting_price = mdl.calc_spot_price_from_reserves(new_base_asset_reserves, new_token_asset_reserves, total_supply, starting_time_remaining)
    starting_apy = mdl.calc_apy_from_reserves(new_base_asset_reserves, new_token_asset_reserves, total_supply, starting_time_remaining, time_stretch)
    starting_hpr = starting_apy * initial_days_remaining / 365
    days_remaining = initial_days_remaining
    calc_hpr = starting_hpr
    while (calc_hpr < initial_hpr):
        day_bump += day_interval
        days_remaining = initial_days_remaining + day_bump
        time_remaining = mdl.days_to_time_remaining(days_remaining, time_stretch)
        spot_price = mdl.calc_spot_price_from_reserves(new_base_asset_reserves, new_token_asset_reserves, total_supply, time_remaining)
        calc_apy = mdl.calc_apy_from_reserves(new_base_asset_reserves, new_token_asset_reserves, total_supply, time_remaining, time_stretch)
        calc_hpr = calc_apy * days_remaining / 365
        # print(f"day_bump: {day_bump}, price: {spot_price:6,.4f}, apy: {calc_apy:6,.4f}, reserves: x={new_base_asset_reserves:6,.0f} y={new_token_asset_reserves:6,.0f}")
        if day_bump>1e6 or day_bump<-1000:
            break
    total_liquidity = new_base_asset_reserves + new_token_asset_reserves * spot_price
    apy = mdl.calc_apy_from_reserves(new_base_asset_reserves, new_token_asset_reserves, total_supply, time_remaining, time_stretch)
    [new_base_asset_reserves_usd, new_token_asset_reserves_usd, total_liquidity_usd] = [x * market_price for x in [new_base_asset_reserves, new_token_asset_reserves, total_liquidity]]
    result.append([trade_in_usd, day_bump, days_remaining, apy, calc_hpr, spot_price, \
        starting_apy, starting_hpr, starting_price, \
        new_base_asset_reserves_usd, new_token_asset_reserves_usd, total_liquidity_usd])
# display(result)

In [ ]:
result_df = pd.DataFrame(result, columns=['trade_in_usd', 'day_bump', 'days_remaining', 'apy', 'hpr', 'spot_price', \
    'starting_apy', 'starting_hpr', 'starting_price', \
    'new_base_asset_reserves_usd', 'new_token_asset_reserves_usd', 'total_liquidity_usd'])
display(result_df.head(5))
display(result_df.tail(5))
num_plots=2
fig, ax = plt.subplots(ncols=1,nrows=num_plots,gridspec_kw={'wspace':0, 'hspace':0.18, 'height_ratios':np.ones(num_plots)})
fig.patch.set_facecolor('white')   # set fig background color to white
a=result_df.plot(
    x='days_remaining',
    y=['apy', 'starting_apy'],
    title='apy vs days remaining',
    ylabel='APY',
    figsize=(10,10),
    grid=True,
    style=['-','--'],
    color=['blue','red'],
    ax=ax[0],
)
a.set_xticklabels([])
result_df.plot(
    x='days_remaining',
    y=['hpr', 'starting_hpr'],
    title='hpr vs days remaining',
    ylabel='HPR',
    figsize=(10,10),
    grid=True,
    style=['-','--'],
    color=['blue','red'],
    ax=ax[1],
)
plt.show()

In [ ]:
[trade_in_usd, day_bump, days_remaining, apy, calc_hpr, spot_price, \
        starting_apy, starting_hpr, starting_price, \
        new_base_asset_reserves_usd, new_token_asset_reserves_usd, total_liquidity_usd] = result[0]
print(f"day_bump: {day_bump}, days_remaining={days_remaining:,.0f}(years={days_remaining/365:.2f}), time_stretch={time_stretch}")
print(f"\
base_asset_reserves:  ${new_base_asset_reserves:6,.0f}\n\
token_asset_reserves: ${new_token_asset_reserves_usd*spot_price:6,.0f} ({new_token_asset_reserves_usd:,.0f} PTs at {spot_price:.4f})\n\
total_liquidity:      ${total_liquidity_usd:6,.0f} ({new_base_asset_reserves_usd+new_token_asset_reserves_usd:,.2f} tokens, base + PT)\n\
                        final  initial  post-swap\n\
APR:                   {apy:6,.2%} "        + f" {initial_apy:6,.2%} "   + f" {starting_apy:6,.2%}\n\
HPR:                   {calc_hpr:6,.2%} "   + f" {initial_hpr:6,.2%} "   + f" {starting_hpr:6,.2%}\n\
price:                 {spot_price:6,.3f} " + f" {initial_price:6,.3f} " + f" {starting_price:6,.3}"
)
print(f"-log(original spot price)/log((new y reserves)/(new x reserves)): {-np.log(initial_price)/np.log((new_token_asset_reserves+total_liquidity_usd)/new_base_asset_reserves)}")
print(f"                                                  *365*t_stretch: {-(np.log(initial_price)/np.log((new_token_asset_reserves+total_liquidity_usd)/new_base_asset_reserves))*(365)*time_stretch}")    

In [ ]:
display(np.concatenate([np.arange(50,1050,50),np.arange(1100,5000,100)]))